In [2]:
import os
import logging
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import numpy as np
import pickle
import json
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
os.listdir(os.getcwd())

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')
seed = 12345
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
class Config:
    def __init__(self):
        self.dataset = 'pheme9'
        self.data_numworker = 4  # 加载数据集时的线程
        self.train_batch = 16  # 训练集的batch大小
        self.test_batch = 16  # 测试集的batch大小
        self.epochs = 500  # 训练迭代次数
        # 词向量
        self.pad_size = 288  # 文本长度 pheme:288
        self.embedding_dim = 100
        self.num_class = 2  # 分类的类别数量
        # gan config
        self.lr_g = 0.001
        self.lr_d = 0.01
        # 训练判别器时，生成数据在判别器上的损失权重
        self.ge_weight = 0.5
        self.re_weight = 0
        # 训练生成器的损失权重
        self.id_weight = 5
        self.cycle_weight = 10
        # generator config
        self.gen_rnn_hidden_size = self.embedding_dim
        self.gen_rnn_num_layers = 2
        self.gen_rnn_dropout = 0.4
        # discriminator config
        self.dis_rnn_hidden_size = self.embedding_dim  # LSTM中隐藏层的大小
        self.dis_rnn_lstm_layers = 2  # RNN中LSTM的层数
        self.dis_rnn_dropout = 0.4  # LSTM中的dropout rate
config = Config()

In [ ]:
def get_device(gpu):
    USE_CUDA = torch.cuda.is_available()
    device = torch.device(gpu if USE_CUDA else 'cpu')
    logging.info("GPU: %s, use %s", USE_CUDA, device)
    return device
config.device = get_device('cuda:0')
print(f"dataset: {config.dataset}", f"pad size: {config.pad_size}", sep='\n-->')

In [ ]:
def load_vocab_and_embedding(dataset):
    embedding_path = f'data/{dataset}_embedding.npz'
    vocab_path = f'data/{dataset}_vocab.pkl'
    with open(vocab_path, 'rb') as f:
        vocab = pickle.load(f)
        vocab_size = len(vocab)
    pretrained_embedding = torch.tensor(np.load(embedding_path, allow_pickle=True)['embeddings'].astype("float32"))
    embedding_dim = pretrained_embedding.size(1)
    print('', f'vocab size: {vocab_size}', f'embedding dim: {embedding_dim}', sep='\n-->')
    return vocab, vocab_size, pretrained_embedding
config.vocab, config.vocab_size, config.pretrained = load_vocab_and_embedding(config.dataset)
config.embedding_dim = config.pretrained.size(1)

In [ ]:
def get_post(content, vocab, pad_size, unk, pad):
    post = []
    for word in content['post'].split(' '):
        idx = vocab.index(word) if word in vocab else unk
        post.append(idx)
    if len(post) < pad_size:
        pad_width = pad_size - len(post)
        post = np.pad(post, (0, pad_width), mode='constant', constant_values=pad)
    elif len(post) > pad_size:
        post = post[:pad_size]
    return post
def get_label(label):
    mask = torch.zeros(2).float()
    mask[int(label)] = 1.0
    return mask

In [ ]:
class buildDataset(data.Dataset):
    def __init__(self, config, mode):
        super(buildDataset, self).__init__()
        self.index = f'data/{config.dataset}_index.pkl'
        self.dataset = f'data/{config.dataset}_data.pkl'
        self.pad_size = config.pad_size  # 句子长度
        self.vocab = config.vocab  # 词表，一个列表，存放着所有的词
        self.unk = config.vocab_size - 2
        self.pad = config.vocab_size - 1
        self.data = []
        with open(self.index, 'rb') as f:
            index = pickle.load(f)[mode]
        with open(self.dataset, 'rb') as f:
            datasets = pickle.load(f)
        """data_index = dataset_index"""
        random.shuffle(index)
        for post in index:
            self.data.append(datasets[post])

    def __getitem__(self, index):
        content = self.data[index]
        post = get_post(content, self.vocab, self.pad_size, self.unk, self.pad)
        label, length, name = content['label'], content['length'], content['post name']
        post = torch.tensor(post).type(torch.int32)
        label = get_label(label)
        return post, label, length, name
    def __len__(self):
        return len(self.data)

In [ ]:
rumor_train_dataset = buildDataset(config, 'rumor_train')
norumor_train_dataset = buildDataset(config, 'norumor_train')
test_dataset = buildDataset(config, 'test')

In [ ]:
def data_collate(batch):
    post = [item[0].numpy() for item in batch]
    label = [item[1].numpy() for item in batch]
    length = [item[2] for item in batch]
    name = [item[3] for item in batch]
    post = torch.tensor(np.array(post), dtype=torch.int)
    label = torch.tensor(np.array(label), dtype=torch.float32)
    return post, label, length, name

In [ ]:
rumor_train_dataloader = data.DataLoader(rumor_train_dataset, batch_size=config.train_batch,num_workers=config.data_numworker, shuffle=True,collate_fn=data_collate, drop_last=True)
norumor_train_dataloader = data.DataLoader(norumor_train_dataset, batch_size=config.train_batch,num_workers=config.data_numworker, shuffle=True,collate_fn=data_collate, drop_last=True)
test_dataloader = data.DataLoader(test_dataset, batch_size=config.test_batch,num_workers=config.data_numworker, shuffle=True,collate_fn=data_collate, drop_last=True)

In [ ]:
def draw_f1(train_f1, test_f1, message, save_path):
    plt.figure(figsize=(16, 8))
    plt.title('f1_score')
    plt.plot(train_f1, label='train f1', linewidth=1, color='red')
    plt.plot(test_f1, label='test f1', linewidth=1, color='black')
    plt.xlabel(message)
    train_max = np.argmax(train_f1)
    test_max = np.argmax(test_f1)
    show_train = '[' + str(train_max) + '  ' + str(train_f1[train_max]) + ']'
    plt.plot(train_max, train_f1[train_max], 'ro')
    plt.annotate(show_train, xy=(train_max, train_f1[train_max]),xytext=(train_max, train_f1[train_max]))
    show_test = '[' + str(test_max) + '  ' + str(test_f1[test_max]) + ']'
    plt.plot(test_max, test_f1[test_max], 'ko')
    plt.annotate(show_test, xy=(test_max, test_f1[test_max]), xytext=(test_max, test_f1[test_max]))
    plt.legend()
    plt.savefig(save_path + '/' + 'f1_score.jpg')
def draw_acc(train_acc, test_acc, message, save_path):
    plt.figure(figsize=(16, 8))
    plt.title('acc')
    plt.plot(train_acc, label='train acc', linewidth=1, color='red')
    plt.plot(test_acc, label='test acc', linewidth=1, color='black')
    plt.xlabel(message)
    train_max = np.argmax(train_acc)
    test_max = np.argmax(test_acc)
    show_train = '[' + str(train_max) + '  ' + str(train_acc[train_max]) + ']'
    plt.plot(train_max, train_acc[train_max], 'ro')
    plt.annotate(show_train, xy=(train_max, train_acc[train_max]),xytext=(train_max, train_acc[train_max]))
    show_test = '[' + str(test_max) + '  ' + str(test_acc[test_max]) + ']'
    plt.plot(test_max, test_acc[test_max], 'ko')
    plt.annotate(show_test, xy=(test_max, test_acc[test_max]), xytext=(test_max, test_acc[test_max]))
    plt.legend()
    plt.savefig(save_path + '/' + 'acc.jpg')
def draw_target(train_acc, test_acc, train_f1, test_f1, name, message):
    current_time = str(get_current_time())
    save_path = f"result/{name}"
    if not os.path.isdir(save_path): os.mkdir(save_path)
    save_path = save_path + "/" + current_time
    if not os.path.isdir(save_path): os.mkdir(save_path)
    draw_acc(train_acc, test_acc, message, save_path)
    draw_f1(train_f1, test_f1, message, save_path)
    return save_path

In [ ]:
def get_scheduler(opt, sc_mode='min', sc_factor=0.8, sc_patience=10, sc_verbose=True, sc_threshold=0.0001,sc_threshold_mode='rel', sc_cooldown=0, sc_min_lr=0, sc_eps=1e-8):
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode=sc_mode, factor=sc_factor, patience=sc_patience,verbose=sc_verbose, threshold=sc_threshold,
                                                           threshold_mode=sc_threshold_mode, cooldown=sc_cooldown,min_lr=sc_min_lr, eps=sc_eps)
    return scheduler
def neg_label(label):
    b = torch.ones_like(label)
    label = b + torch.neg(label)
    return label
class WrongLabelLogger:
    def __init__(self):
        self.logger = {}
    def log(self, label_index, out_index, data):
        wrong_index = label_index - out_index
        wrong_index.astype(int)
        for index, label in enumerate(wrong_index):
            if label != 0:
                if data[index] not in self.logger: self.logger[data[index]] = 1
                else: self.logger[data[index]] += 1
    def write(self, path):
        path = path + '/logger.json'
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(self.logger, f, indent=2, ensure_ascii=False)

In [ ]:
class Generator(nn.Module):
    def __init__(self, config):
        super(Generator, self).__init__()
        self.config = config
        self.lstm = nn.LSTM(config.embedding_dim, config.gen_rnn_hidden_size, config.gen_rnn_num_layers, bias=True, bidirectional=True, batch_first=True, dropout=config.gen_rnn_dropout).to(self.config.device)
        self.fc = nn.Linear(config.gen_rnn_hidden_size * 2, config.embedding_dim).to(self.config.device)
    def forward(self, x, state=None):
        batch, _, _ = x.size()
        if state is None:
            h = torch.randn(self.config.gen_rnn_num_layers * 2, batch, self.config.gen_rnn_hidden_size).float().to(self.config.device)
            c = torch.randn(self.config.gen_rnn_num_layers * 2, batch, self.config.gen_rnn_hidden_size).float().to(self.config.device)
        else: h, c = state
        out, state = self.lstm(x, (h, c))
        out = self.fc(out)
        return out

In [ ]:
class Attention(nn.Module):
    def __init__(self, embedding_dim, device):
        super(Attention, self).__init__()
        self.tanh = nn.Tanh()
        self.w = nn.Parameter(torch.randn(embedding_dim, requires_grad=True)).to(device)
        self.w.data.normal_(mean=0.0, std=0.05)
    def forward(self, x):
        M = self.tanh(x)
        alpha = F.softmax(torch.matmul(M, self.w), dim=1).unsqueeze(-1)
        out = x * alpha
        return out
class Discriminator(nn.Module):
    def __init__(self, config):
        super(Discriminator, self).__init__()
        self.config = config
        self.embedding = nn.Embedding.from_pretrained(config.pretrained, freeze=False).to(config.device)
        self.att = Attention(config.embedding_dim, config.device)
        self.lstm = nn.LSTM(config.embedding_dim * 2, config.dis_rnn_hidden_size, config.dis_rnn_lstm_layers, bias=True, bidirectional=True, batch_first=True, dropout=config.dis_rnn_dropout).to(config.device)
        self.fc = nn.Linear(config.dis_rnn_hidden_size * 2, config.num_class).to(config.device)
    def forward(self, emb, state=None):
        batch, pad_size, embedding_size = emb.size()
        if state is None:
            h = torch.randn(self.config.dis_rnn_lstm_layers * 2, batch, self.config.dis_rnn_hidden_size).float().to(self.config.device)
            c = torch.randn(self.config.dis_rnn_lstm_layers * 2, batch, self.config.dis_rnn_hidden_size).float().to(self.config.device)
        else: h, c = state
        out = self.att(emb)
        out = torch.cat((out, emb), -1)
        out, state = self.lstm(out, (h, c))
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
class WCGan(nn.Module):
    def __init__(self, config):
        super(WCGan, self).__init__()
        self.config = config
        self.mes = f'dataset: {config.dataset}  pad size: {config.pad_size}  embedding dim: {config.embedding_dim}  dis weight:[{config.ge_weight}, {config.re_weight}]  gen weight:[{config.id_weight}, {config.cycle_weight}]'
        print('\n'.join(self.mes.split('  ')))

        self.embedding = nn.Embedding.from_pretrained(config.pretrained, freeze=False).to(self.config.device)
        self.G_r = Generator(self.config)   # 谣言生成器
        self.G_n = Generator(self.config)   # 非谣言生成器
        self.D = Discriminator(self.config) # 判别器

        self.adv_loss = nn.BCEWithLogitsLoss().to(self.config.device)
        # 优化器
        self.G_n_optimizer = torch.optim.RMSprop(self.G_n.parameters(), lr=config.lr_g)
        self.G_r_optimizer = torch.optim.RMSprop(self.G_r.parameters(), lr=config.lr_g)
        self.D_optimizer = torch.optim.RMSprop(self.D.parameters(), lr=config.lr_d)

        self.train_acc, self.test_acc = [], []
        self.train_f1, self.test_f1 = [], []
        self.loss_d, self.loss_gr, self.loss_gn, self.loss_g = 0, 0, 0, 0

        self.wrong_label_logger = WrongLabelLogger()

    def forward(self, norumor_train_dataloader, rumor_train_dataloader, test_dataloader):
        scheduler_D = get_scheduler(self.D_optimizer)
        scheduler_Gr = get_scheduler(self.G_r_optimizer, sc_patience=8)
        scheduler_Gn = get_scheduler(self.G_n_optimizer, sc_patience=8)
        for epoch in range(self.config.epochs):
            print(f'----------------------------------Epoch : {epoch + 1}----------------------------------')
            loop = tqdm(enumerate(zip(norumor_train_dataloader, rumor_train_dataloader)), total=min(len(norumor_train_dataloader), len(rumor_train_dataloader)))
            loop.set_description(f"Training--[Epoch {epoch + 1} : {self.config.epochs}]")
            predict, true = np.array([]), np.array([])
            flag = torch.randint(0, 2, [1])  # 0:norumor 1:rumor
            for i, data in loop:
                flag = (flag + 1) % 2
                or_post, or_label = data[flag][0].to(self.config.device), data[flag][1].to(self.config.device)
                or_post = self.embedding(or_post)  # embedding，将词序列转换成词向量
                if i % 3 != 0: self.train_discriminator(flag, or_post, or_label)
                else: self.train_generator(flag, data)
                with torch.no_grad():
                    out = self.D(or_post)
                true_label = torch.argmax(or_label, 1).cpu().numpy()
                out_label = torch.argmax(out, 1).cpu().numpy()
                true = np.append(true, true_label)
                predict = np.append(predict, out_label)
                self.wrong_label_logger.log(true_label, out_label, data[flag][3])

                acc = metrics.accuracy_score(true, predict)
                f1 = metrics.f1_score(true, predict, average=None)
                f1_s = metrics.f1_score(true, predict, average='macro')
                loop.set_postfix(acc=format(acc, '.5f'), f1=f'[{f1_s}, {f1}]', loss="[{:.4f}|{:.4f}]".format(self.loss_d, self.loss_g))
            test_acc, test_f1, test_loss = self.test(test_dataloader, epoch)
            self.train_acc.append(acc)
            self.test_acc.append(test_acc)
            self.train_f1.append(f1_s)
            self.test_f1.append(test_f1)

            scheduler_D.step(self.loss_d)
            scheduler_Gr.step(self.loss_gr)
            scheduler_Gn.step(self.loss_gn)

        save_path = draw_target(self.train_acc, self.test_acc, self.train_f1, self.test_f1, self.config.model_name, self.mes)
        self.wrong_label_logger.write(save_path)

    def train_discriminator(self, flag, or_post, or_label):
        self.D_optimizer.zero_grad()
        or_out = self.D(or_post)
        or_loss = self.adv_loss(or_out, or_label)
        if flag == 0:  # norumor
            ge_post = self.G_r(or_post)
            re_post = self.G_n(ge_post)
        else:  # rumor
            ge_post = self.G_n(or_post)
            re_post = self.G_r(or_post)
        ge_out = self.D(ge_post)
        ge_loss = self.adv_loss(ge_out, neg_label(or_label))
        re_out = self.D(re_post)
        re_loss = self.adv_loss(re_out, or_label)
        loss_d = or_loss + ge_loss * self.config.ge_weight + re_loss * self.config.re_weight
        loss_d.backward(retain_graph=True)
        self.loss_d = loss_d
        self.D_optimizer.step()

    def train_generator(self, flag, data):
        or_post, or_label = data[flag][0].to(self.config.device), data[flag][1].to(self.config.device)
        or_post = self.embedding(or_post)  # embedding，将词序列转换成词向量
        an_post = data[(flag + 1) % 2][0].to(self.config.device)
        an_post = self.embedding(an_post)
        if flag == 0:  # norumor
            self.G_r_optimizer.zero_grad()
            id_post = self.G_r(an_post)
            ge_post = self.G_r(or_post)
            re_post = self.G_n(ge_post)
        else:  # rumor
            self.G_n_optimizer.zero_grad()
            id_post = self.G_n(an_post)
            ge_post = self.G_n(or_post)
            re_post = self.G_r(ge_post)
        re_out = self.D(re_post)
        # loss_id = -torch.mean(self.D(an_post)) + torch.mean(self.D(id_post))
        loss_id = -torch.mean(an_post) + torch.mean(id_post)
        # loss_id = self.adv_loss(self.D(id_post), neg_label(or_label))
        loss_ge = -torch.mean(self.D(or_post)) + torch.mean(self.D(ge_post))
        loss_cy = self.adv_loss(re_out, or_label)
        # loss_cy = -torch.mean(self.D(or_post)) + torch.mean(self.D(re_post))
        loss_g = loss_ge + loss_id * self.config.id_weight + loss_cy * self.config.cycle_weight
        loss_g.backward(retain_graph=True)
        self.loss_g = loss_g
        self.D.zero_grad()
        if flag == 0:
            self.loss_gr = loss_g
            self.G_n.zero_grad()
            self.G_r_optimizer.step()
        else:
            self.loss_gn = loss_g
            self.G_r.zero_grad()
            self.G_n_optimizer.step()
    def test(self, dataloader, epoch):
        loop = tqdm(enumerate(dataloader), total=len(dataloader))
        loop.set_description(f"Testing---[Epoch {epoch + 1} : {self.config.epochs}]")
        predicts_all, labels_all = np.array([], dtype=int), np.array([], dtype=int)
        loss_all = 0
        with torch.no_grad():
            for i, data in loop:
                post, label = data[0].to(self.config.device), data[1].to(self.config.device)
                post = self.embedding(post)
                out = self.D(post)
                loss = self.adv_loss(out, label)
                loss_all = loss_all + loss
                labels = torch.argmax(label, 1).data.cpu().numpy()
                predicts = torch.argmax(out.data, 1).cpu().numpy()
                labels_all = np.append(labels_all, labels)
                predicts_all = np.append(predicts_all, predicts)
                self.wrong_label_logger.log(labels, predicts, data[3])
                acc = metrics.accuracy_score(labels_all, predicts_all)
                f1 = metrics.f1_score(labels_all, predicts_all, average=None)
                f1_s = metrics.f1_score(labels_all, predicts_all, average='macro')
                total_loss = (loss_all / i).item()

                loop.set_postfix(acc=format(acc, '.5f'), f1=f'[{f1_s}, {f1}]',test_loss=format(total_loss, '.4f'))
        return acc, f1_s, total_loss

In [ ]:
model = WCGan(config)

In [ ]:
model(norumor_train_dataloader, rumor_train_dataloader, test_dataloader)